In [1]:
import pandas as pd

df = pd.read_csv('/content/univ-length-over10.short-col-names.csv')

#df=df.drop(columns = ['date'])
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
#df = imputer.fit_transform(df)

from sklearn.preprocessing import LabelEncoder
le1= LabelEncoder()
le2= LabelEncoder()
le3= LabelEncoder()
df['predWillFollowIncreaseCapacity']=le1.fit_transform(df['predWillFollowIncreaseCapacity'])
df['Ft-ULen-Over10-didFollowIncreaseCapacity']=le2.fit_transform(df['Ft-ULen-Over10-didFollowIncreaseCapacity'])

'''
df=df.dropna()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
'''


#df['icon']= le.fit_transform(df['icon'])

y = df['Ft-ULen-Over10-didFollowIncreaseCapacity'].values
df=df.drop(columns = ['Ft-ULen-Over10-didFollowIncreaseCapacity'])
X = df.values


In [2]:
X=X[1:]
y=y[1:]

for i in range(len(y)):
    if y[i]==2:
        y[i]=1

In [3]:

y=y.reshape(-1, 1)


import numpy as np
X=np.concatenate((X[1:], y[0:-1]), axis=1)

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, y[1:], test_size=0.2, shuffle=False, random_state=42)


In [6]:
x_train1 = x_train1.reshape((x_train1.shape[0], x_train1.shape[1], 1))
x_test1 = x_test1.reshape((x_test1.shape[0], x_test1.shape[1], 1))

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [9]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    #outputs = layers.Dense(1)(x)


    return keras.Model(inputs, outputs)

In [10]:


input_shape = x_train1.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)


model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)
'''
model.compile(
    loss="mean_absolute_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["mean_absolute_error"],
)
'''
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 1)]      0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 20, 1)       2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 20, 1)       7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

In [11]:


callbacks = [keras.callbacks.EarlyStopping(patience=700, restore_best_weights=True)]

history=model.fit(
    x_train1,
    y_train1,
    validation_data=(x_test1, y_test1),
    epochs=2000,
    batch_size=128,
    callbacks=callbacks,
)


Epoch 1/2000
4/4 [==============================] - 14s 375ms/step - loss: 0.7436 - sparse_categorical_accuracy: 0.5053 - val_loss: 0.6275 - val_sparse_categorical_accuracy: 0.6639
Epoch 2/2000
4/4 [==============================] - 0s 45ms/step - loss: 0.6530 - sparse_categorical_accuracy: 0.6385 - val_loss: 0.5979 - val_sparse_categorical_accuracy: 0.6639
Epoch 3/2000
4/4 [==============================] - 0s 40ms/step - loss: 0.6478 - sparse_categorical_accuracy: 0.6575 - val_loss: 0.5833 - val_sparse_categorical_accuracy: 0.6891
Epoch 4/2000
4/4 [==============================] - 0s 40ms/step - loss: 0.6078 - sparse_categorical_accuracy: 0.6786 - val_loss: 0.5788 - val_sparse_categorical_accuracy: 0.6891
Epoch 5/2000
4/4 [==============================] - 0s 44ms/step - loss: 0.6102 - sparse_categorical_accuracy: 0.6808 - val_loss: 0.5728 - val_sparse_categorical_accuracy: 0.6891
Epoch 6/2000
4/4 [==============================] - 0s 40ms/step - loss: 0.5978 - sparse_categorical_ac

In [14]:
#testing prediction
ypred1=model.predict(x_test1)
ypred=[]
#testing prediction
for i in ypred1:
    if i[0]>i[1]:
        ypred.append(0)
    else:
        ypred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(ypred, y_test1))



print('f1_score', f1_score(ypred, y_test1))



print('jaccard_score', jaccard_score(ypred, y_test1))
print('roc_auc_score', roc_auc_score(ypred, y_test1))
print('precision_score', precision_score(ypred, y_test1))

print('recall_score', recall_score(ypred, y_test1))


accuracy 0.773109243697479
f1_score 0.7157894736842105
jaccard_score 0.5573770491803278
roc_auc_score 0.7626173708920189
precision_score 0.723404255319149
recall_score 0.7083333333333334


In [15]:
#ttraining prediction
xpred1=model.predict(x_train1)
xpred=[]

for i in xpred1:
    if i[0]>i[1]:
        xpred.append(0)
    else:
        xpred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(xpred, y_train1))



print('f1_score', f1_score(xpred, y_train1))



print('jaccard_score', jaccard_score(xpred, y_train1))
print('roc_auc_score', roc_auc_score(xpred, y_train1))
print('precision_score', precision_score(xpred, y_train1))

print('recall_score', recall_score(xpred, y_train1))


accuracy 0.9006342494714588
f1_score 0.8613569321533924
jaccard_score 0.7564766839378239
roc_auc_score 0.9182263173049237
precision_score 0.776595744680851
recall_score 0.9668874172185431
